In [1]:
from transformers import AutoImageProcessor, AutoModel

processor = AutoImageProcessor.from_pretrained("StanfordAIMI/dinov2-base-xray-224")
backbone = AutoModel.from_pretrained("StanfordAIMI/dinov2-base-xray-224")
feat_dim = backbone.config.hidden_size

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
import sys
sys.path.append('..')
from FusionModels.MLP.MLP import MLP
from wrapperModul import wrapperModel

In [3]:
head_mlp = MLP([1024, 2048, 2])

In [4]:
for p in head_mlp.parameters():
    p.requires_grad = True

In [5]:
for p in backbone.parameters():
    p.requires_grad = False

In [6]:
model = wrapperModel(backbone, head_mlp)

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split

import sys
sys.path.append('..')  
from DataLoader.dataLoader import IntubiertDataset
from Train.trainLoop import train_model

from torchvision import transforms
from torch.utils.data import DataLoader

import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [8]:
DATA_INFO_FILE = r"../Data/PatientInfo/splittedPatientData.xlsx"
IMAGE_PATH = r"../Data/Patient_Images"
LOG_PATH = r"Logs/dinov2-base-xray-224HeadTune.log"

In [9]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),                             
    transforms.ToTensor(),                                        
])


val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [10]:
train_dataset = IntubiertDataset(IMAGE_PATH,DATA_INFO_FILE , "train",transforms=train_transform)
val_dataset = IntubiertDataset(IMAGE_PATH, DATA_INFO_FILE, "val",transforms=val_transform)
test_dataset = IntubiertDataset(IMAGE_PATH, DATA_INFO_FILE, "test", transforms=test_transform)

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=128, num_workers=20)
val_dataloader = DataLoader(val_dataset, batch_size=128, num_workers=20)
test_dataloader = DataLoader(test_dataset, batch_size=128, num_workers=20)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [13]:
scheduler = ReduceLROnPlateau(
    optimizer,          
    mode='min',          
    factor=0.5,          
    patience=2,          
    min_lr=1e-6,        
    cooldown=0,          
    threshold=1)

In [14]:
train_model(model, train_dataloader, val_dataloader, 'cuda', criterion, optimizer, 'best_model.pth', LOG_PATH,scheduler=scheduler)


Training completed!


In [15]:
import timm
import torch

model.to('cuda')

checkpoint = torch.load('best_model.pth', weights_only=True)

model.load_state_dict(checkpoint, strict=False)

print("Model loaded successfully with non-strict loading")

Model loaded successfully with non-strict loading


In [16]:
from Test_Phrease.Eval.testEvaluate import evaluate_model_on_testset

test_results=evaluate_model_on_testset(model, 'dinov2-base-xray-224HeadTune', test_dataloader, 'cuda', criterion, 'testeval')


Test Loss: 0.5191
Accuracy: 0.7308, F1: 0.5297, AUC: 0.7986
Precision: 1.0000, Recall: 0.1250, Specificity: 1.0000
Confusion Matrix:
[[18  0]
 [ 7  1]]


In [17]:
from Test_Phrease.Eval_Results.concatModelTestResults import save_metrics_to_excel

save_metrics_to_excel(test_results, '../ModelResultsDocs/testresults.xlsx')

ModuleNotFoundError: No module named 'Test_Phrease.Eval_Results'